In [1]:
import cv2 as cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import time
import os
import matplotlib.pyplot as plt

In [2]:
mp_holistic = mp.solutions.holistic
mp_pose=mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)           # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)               # Draw pose connections
    #mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)          # Draw left hand connections
    #mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
   
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    

In [6]:
def get_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)

In [7]:
out = np.array(['good left half','good right half','bad left half', 'bad right half'])
sq_len=20
#fps_a= cap.get(cv2.CAP_PROP_FPS)
#frame_count=cap.get(cv2.CAP_PROP_FRAME_COUNT)
#no_sq=round(frame_count/fps_a)
no_sq=4

In [8]:
vid_q=['lr.mp4','rl1.mp4','rl2.mp4']

In [9]:
cap = cv2.VideoCapture('lr.mp4')
# used to record the time when we processed last frame
prev_frame_time = 0

# used to record the time at which we processed current frame
new_frame_time = 0
# VIDEO FEED

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    slope_shoulder_l,slope_hip_l,rate_slope_shoulder_l,rate_slope_shoulder_length_l=[],[],[],[]

    differenceHipsList = []
    differenceShouldersList = []
    leftshoulderList = []
    rightshoulderList = []
    lsp=[]
    rsp=[]
    lhp=[]
    rhp=[]
    lefthipList = []
    righthipList = []
    midpointsholder_l=[]
    midpointhip_l=[]
    spinelength_l=[]
    s_len=[]
    h_len=[]
    ratio_l=[]
    framecounter = 0
    while cap.isOpened():

        ret, frame = cap.read()
        if not ret:
            break
        if frame is not None:
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            image.flags.writeable = False
            results = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)


            font = cv2.FONT_HERSHEY_SIMPLEX
            new_frame_time = time.time()
        # Calculating the fps

        # fps will be number of frame processed in given time frame
        # since their will be most of time error of 0.001 second
        # we will be subtracting it to get more accurate result
            fps = 1 / (new_frame_time - prev_frame_time)
            prev_frame_time = new_frame_time

        # converting the fps into integer
            fps = int(fps)

        # converting the fps to string so that we can display it on frame
        # by using putText function
            fps_1 = str(fps)

        # putting the FPS count on the frame
            cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



            try:
                landmarks = results.pose_landmarks.landmark
                leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                differenceHips = np.absolute(righthip.z - lefthip.z)
                differenceHipsList.append(differenceHips)
                differenceShouldersList.append(differenceShoulders)
                leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                lefthipList.append([lefthip.x, lefthip.y])
                righthipList.append([righthip.x, righthip.y])

            except:

                pass




            differenceHipsAverage = np.mean(differenceHipsList)
            differenceShouldersAverage = np.mean(differenceShouldersList)


            slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
            slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))

            rate_slope_shoulder = slope_shoulder * fps

            rate_slope_shoulder_length = differenceShouldersAverage * fps
            midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
            midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
            leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
            rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
            lefthip_poin=np.array([lefthip.x,lefthip.y])
            righthip_poin=np.array([righthip.x,righthip.y])
            midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
            midpointhip_p_x = (lefthip.x+righthip.x)/2
            midpointhip_p_y = (lefthip.y+righthip.y)/2
            midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])

            s_len_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)
            h_len_v=np.linalg.norm(lefthip_poin-righthip_poin)
            ratio_v=s_len_v/h_len_v
            spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)
            framecounter += 1



        #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
        #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
        #From there, we should be able to detect in which phase the subject is in each frame.

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                    )


            cv2.imshow('Mediapipe Feed', image)
            #image_l.append(image)
            #print(image.shape)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y]),slope_shoulder_l.append(slope_shoulder),slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length),s_len.append(s_len_v),h_len.append(h_len_v)

cap.release()
#print(midpointhip_l)
cv2.destroyAllWindows()

In [10]:
data={'Left_Shoulder':lsp,
      'Right_Shoulder':rsp,
      'Left_Hip':lhp,
      'Right_Hip':rhp,
      'difference_shoulder': differenceShouldersList,
      'difference_hip': differenceHipsList,
      'Midpoint_Shoulder':midpointsholder_l,
      'Midpoint_Hip':midpointhip_l,
      'Spine_Length':spinelength_l,
      'Shoulder_Length':s_len,
      'Hip_Length':h_len,
      'Ratio_of_shoulder/hip':ratio_l,
      'Slope_shoulder':slope_shoulder_l,
      'Slope_hip':slope_hip_l,
      'Rate_of_Shoulder':rate_slope_shoulder_l,
      'Rate_of_hip':rate_slope_shoulder_length_l
      }
dataF1=pd.DataFrame(data)


In [11]:
data={'difference_shoulder': differenceShouldersList,
      'difference_hip': differenceHipsList,
      'Spine_Length':spinelength_l,
      'Shoulder_Length':s_len,
      'Slope_shoulder':slope_shoulder_l,
      'Slope_hip':slope_hip_l
      
        }
dataF12=pd.DataFrame(data)

In [12]:
dataF12

,difference_shoulder,difference_hip,Spine_Length,Shoulder_Length,Slope_shoulder,Slope_hip
0,0.014243,0.059451,0.207074,0.082371,-0.179524,-0.273098
1,0.047232,0.089859,0.203403,0.079959,-0.137879,-0.185912
2,0.037879,0.089142,0.203571,0.079026,-0.153273,-0.177371
3,0.063292,0.094772,0.204015,0.078888,-0.172748,-0.172522
4,0.065822,0.099830,0.205028,0.078558,-0.183343,-0.186418
5,0.056512,0.098435,0.206364,0.078403,-0.196871,-0.207929
6,0.048758,0.098339,0.207902,0.078432,-0.215174,-0.245699
7,0.050069,0.097904,0.209284,0.078580,-0.222615,-0.268594
8,0.048497,0.096327,0.210895,0.079137,-0.225086,-0.290829
9,0.045598,0.095941,0.212240,0.079622,-0.206556,-0.305865


In [13]:
cap = cv2.VideoCapture('rl1.mp4')
# used to record the time when we processed last frame
prev_frame_time = 0

# used to record the time at which we processed current frame
new_frame_time = 0
# VIDEO FEED

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    slope_shoulder_l,slope_hip_l,rate_slope_shoulder_l,rate_slope_shoulder_length_l=[],[],[],[]

    differenceHipsList = []
    differenceShouldersList = []
    leftshoulderList = []
    rightshoulderList = []
    lsp=[]
    rsp=[]
    lhp=[]
    rhp=[]
    lefthipList = []
    righthipList = []
    midpointsholder_l=[]
    midpointhip_l=[]
    spinelength_l=[]
    s_len=[]
    h_len=[]
    ratio_l=[]
    framecounter = 0
    while cap.isOpened():

        ret, frame = cap.read()
        if not ret:
            break
        if frame is not None:
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            image.flags.writeable = False
            results = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)


            font = cv2.FONT_HERSHEY_SIMPLEX
            new_frame_time = time.time()
        # Calculating the fps

        # fps will be number of frame processed in given time frame
        # since their will be most of time error of 0.001 second
        # we will be subtracting it to get more accurate result
            fps = 1 / (new_frame_time - prev_frame_time)
            prev_frame_time = new_frame_time

        # converting the fps into integer
            fps = int(fps)

        # converting the fps to string so that we can display it on frame
        # by using putText function
            fps_1 = str(fps)

        # putting the FPS count on the frame
            cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



            try:
                landmarks = results.pose_landmarks.landmark
                leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                differenceShoulders = np.linalg.norm(rightshoulder.z - leftshoulder.z)
                righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                differenceHips = np.linalg.norm(righthip.z - lefthip.z)
                differenceHipsList.append(differenceHips)
                differenceShouldersList.append(differenceShoulders)
                leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                lefthipList.append([lefthip.x, lefthip.y])
                righthipList.append([righthip.x, righthip.y])

            except:

                pass




            differenceHipsAverage = np.mean(differenceHipsList)
            differenceShouldersAverage = np.mean(differenceShouldersList)


            slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
            slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))

            rate_slope_shoulder = slope_shoulder * fps

            rate_slope_shoulder_length = differenceShouldersAverage * fps
            midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
            midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
            leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
            rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
            lefthip_poin=np.array([lefthip.x,lefthip.y])
            righthip_poin=np.array([righthip.x,righthip.y])
            midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
            midpointhip_p_x = (lefthip.x+righthip.x)/2
            midpointhip_p_y = (lefthip.y+righthip.y)/2
            midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])

            s_len_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)
            h_len_v=np.linalg.norm(lefthip_poin-righthip_poin)
            ratio_v=s_len_v/h_len_v
            spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)
            framecounter += 1



        #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
        #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
        #From there, we should be able to detect in which phase the subject is in each frame.

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                    )


            cv2.imshow('Mediapipe Feed', image)
            #image_l.append(image)
            #print(image.shape)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y]),slope_shoulder_l.append(slope_shoulder),slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length),s_len.append(s_len_v),h_len.append(h_len_v)

cap.release()
#print(midpointhip_l)
cv2.destroyAllWindows()

In [14]:
data={'Left_Shoulder':lsp,
      'Right_Shoulder':rsp,
      'Left_Hip':lhp,
      'Right_Hip':rhp,
      'difference_shoulder': differenceShouldersList,
      'difference_hip': differenceHipsList,
      'Midpoint_Shoulder':midpointsholder_l,
      'Midpoint_Hip':midpointhip_l,
      'Spine_Length':spinelength_l,
      'Shoulder_Length':s_len,
      'Hip_Length':h_len,
      'Ratio_of_shoulder/hip':ratio_l,
      'Slope_shoulder':slope_shoulder_l,
      'Slope_hip':slope_hip_l,
      'Rate_of_Shoulder':rate_slope_shoulder_l,
      'Rate_of_hip':rate_slope_shoulder_length_l
      }
dataF2=pd.DataFrame(data)

In [15]:
data={'difference_shoulder': differenceShouldersList,
      'difference_hip': differenceHipsList,
      'Spine_Length':spinelength_l,
      'Shoulder_Length':s_len,
      'Slope_shoulder':slope_shoulder_l,
      'Slope_hip':slope_hip_l
      
        }
dataF22=pd.DataFrame(data)

In [16]:
dataF22

,difference_shoulder,difference_hip,Spine_Length,Shoulder_Length,Slope_shoulder,Slope_hip
0,0.004061,0.026336,0.210720,0.084130,0.294816,0.193998
1,0.004934,0.018692,0.209991,0.083663,0.307558,0.179895
2,0.004892,0.019321,0.210079,0.083248,0.312534,0.175844
3,0.002591,0.027823,0.209404,0.083152,0.311000,0.175435
4,0.006370,0.031092,0.209301,0.083160,0.311936,0.184446
5,0.010775,0.036779,0.209900,0.082822,0.308662,0.192816
6,0.023543,0.044409,0.210175,0.082289,0.308124,0.213433
7,0.030558,0.053107,0.210186,0.082118,0.318238,0.230625
8,0.028733,0.064569,0.211631,0.081773,0.322379,0.244954
9,0.027015,0.069947,0.211639,0.081647,0.326449,0.263553


In [17]:
cap = cv2.VideoCapture('rl1.mp4')
# used to record the time when we processed last frame
prev_frame_time = 0

# used to record the time at which we processed current frame
new_frame_time = 0
# VIDEO FEED

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    slope_shoulder_l,slope_hip_l,rate_slope_shoulder_l,rate_slope_shoulder_length_l=[],[],[],[]

    differenceHipsList = []
    differenceShouldersList = []
    leftshoulderList = []
    rightshoulderList = []
    lsp=[]
    rsp=[]
    lhp=[]
    rhp=[]
    lefthipList = []
    righthipList = []
    midpointsholder_l=[]
    midpointhip_l=[]
    spinelength_l=[]
    s_len=[]
    h_len=[]
    ratio_l=[]
    framecounter = 0
    while cap.isOpened():

        ret, frame = cap.read()
        if not ret:
            break
        if frame is not None:
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            image.flags.writeable = False
            results = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)


            font = cv2.FONT_HERSHEY_SIMPLEX
            new_frame_time = time.time()
        # Calculating the fps

        # fps will be number of frame processed in given time frame
        # since their will be most of time error of 0.001 second
        # we will be subtracting it to get more accurate result
            fps = 1 / (new_frame_time - prev_frame_time)
            prev_frame_time = new_frame_time

        # converting the fps into integer
            fps = int(fps)

        # converting the fps to string so that we can display it on frame
        # by using putText function
            fps_1 = str(fps)

        # putting the FPS count on the frame
            cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



            try:
                landmarks = results.pose_landmarks.landmark
                leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                differenceHips = np.absolute(righthip.z - lefthip.z)
                differenceHipsList.append(differenceHips)
                differenceShouldersList.append(differenceShoulders)
                leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                lefthipList.append([lefthip.x, lefthip.y])
                righthipList.append([righthip.x, righthip.y])

            except:

                pass




            differenceHipsAverage = np.mean(differenceHipsList)
            differenceShouldersAverage = np.mean(differenceShouldersList)


            slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
            slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))

            rate_slope_shoulder = slope_shoulder * fps

            rate_slope_shoulder_length = differenceShouldersAverage * fps
            midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
            midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
            leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
            rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
            lefthip_poin=np.array([lefthip.x,lefthip.y])
            righthip_poin=np.array([righthip.x,righthip.y])
            midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
            midpointhip_p_x = (lefthip.x+righthip.x)/2
            midpointhip_p_y = (lefthip.y+righthip.y)/2
            midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])

            s_len_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)
            h_len_v=np.linalg.norm(lefthip_poin-righthip_poin)
            ratio_v=s_len_v/h_len_v
            spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)
            framecounter += 1



        #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
        #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
        #From there, we should be able to detect in which phase the subject is in each frame.

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                    )


            cv2.imshow('Mediapipe Feed', image)
            #image_l.append(image)
            #print(image.shape)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y]),slope_shoulder_l.append(slope_shoulder),slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length),s_len.append(s_len_v),h_len.append(h_len_v)

cap.release()
#print(midpointhip_l)
cv2.destroyAllWindows()

In [18]:
data={'Left_Shoulder':lsp,
      'Right_Shoulder':rsp,
      'Left_Hip':lhp,
      'Right_Hip':rhp,
      'difference_shoulder': differenceShouldersList,
      'difference_hip': differenceHipsList,
      'Midpoint_Shoulder':midpointsholder_l,
      'Midpoint_Hip':midpointhip_l,
      'Spine_Length':spinelength_l,
      'Shoulder_Length':s_len,
      'Hip_Length':h_len,
      'Ratio_of_shoulder/hip':ratio_l,
      'Slope_shoulder':slope_shoulder_l,
      'Slope_hip':slope_hip_l,
      'Rate_of_Shoulder':rate_slope_shoulder_l,
      'Rate_of_hip':rate_slope_shoulder_length_l
      }
dataF3=pd.DataFrame(data)

In [19]:
data={'difference_shoulder': differenceShouldersList,
      'difference_hip': differenceHipsList,
      'Spine_Length':spinelength_l,
      'Shoulder_Length':s_len,
      'Slope_shoulder':slope_shoulder_l,
      'Slope_hip':slope_hip_l
      
        }
dataF32=pd.DataFrame(data)

In [20]:
dataF32

,difference_shoulder,difference_hip,Spine_Length,Shoulder_Length,Slope_shoulder,Slope_hip
0,0.004061,0.026336,0.210720,0.084130,0.294816,0.193998
1,0.004934,0.018692,0.209991,0.083663,0.307558,0.179895
2,0.004892,0.019321,0.210079,0.083248,0.312534,0.175844
3,0.002591,0.027823,0.209404,0.083152,0.311000,0.175435
4,0.006370,0.031092,0.209301,0.083160,0.311936,0.184446
5,0.010775,0.036779,0.209900,0.082822,0.308662,0.192816
6,0.023543,0.044409,0.210175,0.082289,0.308124,0.213433
7,0.030558,0.053107,0.210186,0.082118,0.318238,0.230625
8,0.028733,0.064569,0.211631,0.081773,0.322379,0.244954
9,0.027015,0.069947,0.211639,0.081647,0.326449,0.263553


In [21]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [22]:
label_map = {label:num for num, label in enumerate(out)}

In [23]:
label_map

{'good left half': 0,
 'good right half': 1,
 'bad left half': 2,
 'bad right half': 3}

In [24]:
a=label_map.items()

In [25]:
a

dict_items([('good left half', 0), ('good right half', 1), ('bad left half', 2), ('bad right half', 3)])

In [26]:
len(label_map)

4

In [27]:
dataFR = [np.array(dataF12),np.array(dataF22),np.array(dataF32),np.array(dataF32)]

In [49]:
seq, labels =[], []
for i in range(len(out)):
    seq.append(dataFR[i])
    seq[i]=seq[i][:30]
    labels.append(label_map[out[i]])

In [50]:
seq=np.array(seq)

In [51]:
seq

array([[[ 1.42431781e-02,  5.94508741e-02,  2.07073635e-01,
          8.23706769e-02, -1.79524129e-01, -2.73098408e-01],
        [ 4.72316071e-02,  8.98590647e-02,  2.03402796e-01,
          7.99594632e-02, -1.37879151e-01, -1.85911794e-01],
        [ 3.78790721e-02,  8.91416930e-02,  2.03571194e-01,
          7.90258819e-02, -1.53273489e-01, -1.77370696e-01],
        [ 6.32918198e-02,  9.47724953e-02,  2.04014763e-01,
          7.88877985e-02, -1.72748329e-01, -1.72522006e-01],
        [ 6.58221235e-02,  9.98304337e-02,  2.05027602e-01,
          7.85582008e-02, -1.83343128e-01, -1.86418316e-01],
        [ 5.65123819e-02,  9.84349810e-02,  2.06364112e-01,
          7.84032852e-02, -1.96871123e-01, -2.07928592e-01],
        [ 4.87576330e-02,  9.83388200e-02,  2.07901761e-01,
          7.84319583e-02, -2.15173886e-01, -2.45699340e-01],
        [ 5.00687966e-02,  9.79040787e-02,  2.09283949e-01,
          7.85801436e-02, -2.22615185e-01, -2.68593671e-01],
        [ 4.84973248e-02,  9.632

In [52]:
seq.shape

(4, 30, 6)

In [53]:
labels

[0, 1, 2, 3]

In [55]:
seq[0].shape

(30, 6)

In [56]:
np.array(labels).shape

(4,)

In [57]:
x = np.array(seq)
y = to_categorical(labels).astype(int)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, shuffle=True, random_state=27)

In [58]:
type(x_test)

numpy.ndarray

In [59]:
y_test.shape

(1, 4)

In [60]:
x_test.shape

(1, 30, 6)

In [61]:
x_test

array([[[ 4.06093895e-03,  2.63362452e-02,  2.10719981e-01,
          8.41299779e-02,  2.94815888e-01,  1.93998237e-01],
        [ 4.93435562e-03,  1.86917456e-02,  2.09991467e-01,
          8.36633246e-02,  3.07557834e-01,  1.79894848e-01],
        [ 4.89173084e-03,  1.93210775e-02,  2.10078691e-01,
          8.32475428e-02,  3.12534467e-01,  1.75843876e-01],
        [ 2.59057432e-03,  2.78232517e-02,  2.09404190e-01,
          8.31523263e-02,  3.11000121e-01,  1.75434537e-01],
        [ 6.37002289e-03,  3.10921278e-02,  2.09300508e-01,
          8.31604511e-02,  3.11936281e-01,  1.84445811e-01],
        [ 1.07745230e-02,  3.67786754e-02,  2.09900465e-01,
          8.28217529e-02,  3.08662064e-01,  1.92816308e-01],
        [ 2.35425159e-02,  4.44085058e-02,  2.10174994e-01,
          8.22887971e-02,  3.08124064e-01,  2.13432996e-01],
        [ 3.05576958e-02,  5.31073790e-02,  2.10185633e-01,
          8.21175630e-02,  3.18238242e-01,  2.30625457e-01],
        [ 2.87328549e-02,  6.456

In [62]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [63]:
#log_dir = os.path.join('Logs')
#tb_callback = EarlyStopping(monitor='val_loss', patience=30, mode='min', restore_best_weights=True)
#e_callback = TensorBoard(log_dir=log_dir)
 

In [64]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,6)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
# model.add(Dropout(0.12))
# model.add(LSTM(256,return_sequences=True, activation='relu'))  #added layer
# model.add(LSTM(128, return_sequences=True, activation='relu')) #added layer
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(out.shape[0], activation='softmax'))

In [65]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            18176     
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 4)                 132       
                                                                 
Total params: 172,772
Trainable params: 172,772
Non-tr

In [66]:
model.compile(optimizer='RMSProp', loss='categorical_crossentropy', metrics=['accuracy'])

In [68]:
history = model.fit(x_train, y_train, epochs=1000, shuffle = True)

Epoch 1/1000
1/1 [==============================] - 0s 33ms/step - loss: 10.8513 - accuracy: 0.6667
Epoch 2/1000
1/1 [==============================] - 0s 33ms/step - loss: 1.0888 - accuracy: 0.6667
Epoch 3/1000
1/1 [==============================] - 0s 32ms/step - loss: 0.8158 - accuracy: 0.6667
Epoch 4/1000
1/1 [==============================] - 0s 30ms/step - loss: 0.7912 - accuracy: 0.6667
Epoch 5/1000
1/1 [==============================] - 0s 34ms/step - loss: 0.7710 - accuracy: 0.6667
Epoch 6/1000
1/1 [==============================] - 0s 32ms/step - loss: 0.7464 - accuracy: 0.6667
Epoch 7/1000
1/1 [==============================] - 0s 32ms/step - loss: 0.7139 - accuracy: 0.6667
Epoch 8/1000
1/1 [==============================] - 0s 32ms/step - loss: 0.6714 - accuracy: 0.6667
Epoch 9/1000
1/1 [==============================] - 0s 32ms/step - loss: 0.6169 - accuracy: 0.6667
Epoch 10/1000
1/1 [==============================] - 0s 32ms/step - loss: 0.5531 - accuracy: 0.6667
Epoch 11